In [1]:
DIR = "timing/";

|-----------|




### Basic Framework

In [2]:
PATH_TO_GAPSE = "../";

In [3]:
include(PATH_TO_GAPSE * "src/GaPSE.jl");

In [4]:
using DelimitedFiles, Plots, Dierckx, SpecialFunctions
using Printf, LaTeXStrings, Turing, StatsPlots, GridInterpolations

In [5]:
z_min, z_max, θ_max = 0.05, 0.20, π / 2.0
#z_min, z_max, θ_max = 1.0, 1.5, π / 2.0

WFI_opts = Dict(
    :ss_start => 0.0, :ss_stop => 0.0, 
    :ss_step => 50, :llim => 0.0, :rlim => Inf, 
    :rtol => 5e-2, :atol => 0.0, :N => 1000, #:pr => true,
)

FILE_F_MAP = PATH_TO_GAPSE * "data/F_REFERENCE_pi2.txt"
FILE_PS = PATH_TO_GAPSE * "test/datatest/file_pk.txt"
FILE_BACKGROUND = PATH_TO_GAPSE * "test/datatest/WideA_ZA_background.dat"

params = GaPSE.CosmoParams(z_min, z_max, θ_max;
    Ω_b = 0.0489, Ω_cdm = 0.251020, h_0 = 0.70, s_lim=1e-2,
    s_b = 0.0, 𝑓_evo = 0.0, b = 1.0,
    IPS_opts = Dict(
        :fit_left_min => 1e-6, :fit_left_max => 3e-6,
        :fit_right_min => 1e1, :fit_right_max => 2e1),
    IPSTools_opts = Dict(
        :N => 1024, :fit_min => 0.05, :fit_max => 0.5, 
        :con => true, :k_min => 1e-8, :k_max => 10.0),
    WFI_opts = WFI_opts,
);

In [6]:
#=
new_calc_μs = union([μ for μ in -1.0:0.01:(-0.9)], 
        [μ for μ in -0.9:0.05:0.9], 
        [μ for μ in 0.9:0.01:1.0]);
FILE_IF_MAP = PATH_TO_GAPSE*"data/IntF_pi2_z115.txt"

GaPSE.print_map_IntegratedF(
    FILE_F_MAP, FILE_IF_MAP, 
    z_min, z_max, new_calc_μs, FILE_BACKGROUND;
    trap = true, WFI_opts...
)
=#

In [7]:
#FILE_IF_MAP = PATH_TO_GAPSE*"data/IntF_pi2_z115.txt"
FILE_IF_MAP = PATH_TO_GAPSE*"data/IntegrF_REFERENCE_pi2_z005020.txt"
cosmo = GaPSE.Cosmology(params, FILE_BACKGROUND, FILE_PS, FILE_F_MAP, FILE_IF_MAP);

GaPSE.parameters_used(stdout, cosmo)

###############
#    GaPSE    #
############### 
#
# The Cosmology considered had the following paremeters:
# 
# - Matter Power Spectrum input file: "../test/datatest/file_pk.txt"
# - Background data input file: "../test/datatest/WideA_ZA_background.dat"
# - F window function input file: "../data/F_REFERENCE_pi2.txt"
# - Integrated F window function input file: "../data/IntegrF_REFERENCE_pi2_z005020.txt"
#
# - Basic CosmoParams considered: 
#	 z_min = 0.05 	 z_max = 0.2
#	 θ_max = 1.5707963267948966 [rad] 	 h_0 = 0.7
#	 Ω_b = 0.0489 	 Ω_cdm = 0.25102 	 Ω_M0 = 0.29992
#	 b = 1.0 	 f_evo = 0.0 	 s_b = 0.0
#
# - CosmoParams about the Input Power Spectrum: 
#	 fit_left_min = 1.0e-6 	 fit_right_min = 10.0 	 
#	 fit_left_max = 3.0e-6 	 fit_right_max = 20.0 	 
#
# - CosmoParams about the Input Power Spectrum Tools: 
#	 k_max = 10.0 	 fit_max = 0.5 	 N = 1024 	 
#	 con = true 	 k_min = 1.0e-8 	 fit_min = 0.05 	 
#
# - CosmoParams about the Integrated Window Function F: 
#	 ss_step = 50 	 N = 1

### noF L=0

In [14]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L0_noF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

auto_newton, L=0: 100%|█████████████████████████████████| Time: 0:00:01



time needed for map_ξ_GNC_multipole for auto_newton [in s] = 1.18947 



In [17]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L0_noF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

auto_lensing, L=0: 100%|████████████████████████████████| Time: 0:09:35



time needed for map_ξ_GNC_multipole for auto_lensing [in s] = 575.86794 



In [8]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L0_noF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = false, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

auto_newton, L=0: 100%|█████████████████████████████████| Time: 0:00:01
auto_doppler, L=0: 100%|████████████████████████████████| Time: 0:00:00
auto_lensing, L=0: 100%|████████████████████████████████| Time: 0:09:17
auto_localgp, L=0: 100%|████████████████████████████████| Time: 0:00:00
auto_integratedgp, L=0: 100%|███████████████████████████| Time: 0:03:23
newton_doppler, L=0: 100%|██████████████████████████████| Time: 0:00:00
doppler_newton, L=0: 100%|██████████████████████████████| Time: 0:00:00
newton_lensing, L=0: 100%|██████████████████████████████| Time: 0:00:27
lensing_newton, L=0: 100%|██████████████████████████████| Time: 0:00:26
newton_localgp, L=0: 100%|██████████████████████████████| Time: 0:00:00
localgp_newton, L=0: 100%|██████████████████████████████| Time: 0:00:00
newton_integratedgp, L=0: 100%|█████████████████████████| Time: 0:00:28
integratedgp_newton, L=0: 100%|█████████████████████████| Time: 0:00:28
lensing_doppler, L=0: 100%|█████████████████████████████| Time: 


time needed for map_sum_ξ_GNC_multipole [in s] = 1654.97374 



### noF L=2

In [12]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L2_noF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

auto_newton, L=2: 100%|█████████████████████████████████| Time: 0:00:00



time needed for map_ξ_GNC_multipole for auto_newton [in s] = 0.45877 



In [13]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L2_noF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

auto_lensing, L=2: 100%|████████████████████████████████| Time: 0:09:23



time needed for map_ξ_GNC_multipole for auto_lensing [in s] = 563.94554 



In [14]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L2_noF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = false, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

auto_newton, L=2: 100%|█████████████████████████████████| Time: 0:00:00
auto_doppler, L=2: 100%|████████████████████████████████| Time: 0:00:00
auto_lensing, L=2: 100%|████████████████████████████████| Time: 0:09:09
auto_localgp, L=2: 100%|████████████████████████████████| Time: 0:00:00
auto_integratedgp, L=2: 100%|███████████████████████████| Time: 0:03:28
newton_doppler, L=2: 100%|██████████████████████████████| Time: 0:00:00
doppler_newton, L=2: 100%|██████████████████████████████| Time: 0:00:00
newton_lensing, L=2: 100%|██████████████████████████████| Time: 0:00:26
lensing_newton, L=2: 100%|██████████████████████████████| Time: 0:00:26
newton_localgp, L=2: 100%|██████████████████████████████| Time: 0:00:00
localgp_newton, L=2: 100%|██████████████████████████████| Time: 0:00:00
newton_integratedgp, L=2: 100%|█████████████████████████| Time: 0:00:28
integratedgp_newton, L=2: 100%|█████████████████████████| Time: 0:00:27
lensing_doppler, L=2: 100%|█████████████████████████████| Time: 


time needed for map_sum_ξ_GNC_multipole [in s] = 1627.55057 



### noF L=4

In [18]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L4_noF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=4,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

auto_newton, L=4: 100%|█████████████████████████████████| Time: 0:00:00



time needed for map_ξ_GNC_multipole for auto_newton [in s] = 0.46736 



In [19]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L4_noF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=4,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

auto_lensing, L=4: 100%|████████████████████████████████| Time: 0:09:33



time needed for map_ξ_GNC_multipole for auto_lensing [in s] = 573.76925 



In [ ]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L4_noF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = false, enhancer = 1e8, L=4,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

auto_newton, L=4: 100%|█████████████████████████████████| Time: 0:00:00
auto_doppler, L=4: 100%|████████████████████████████████| Time: 0:00:00
auto_lensing, L=4:  85%|███████████████████████████▎    |  ETA: 0:01:27

### with F L=0

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L0_withF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = true, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L0_withF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = true, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

In [ ]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L0_withF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = true, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

### with F L=2

In [15]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L2_withF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = true, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

auto_newton, L=2: 100%|█████████████████████████████████| Time: 0:00:13



time needed for map_ξ_GNC_multipole for auto_newton [in s] = 13.15246 



In [16]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L2_withF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = true, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

auto_lensing, L=2: 100%|████████████████████████████████| Time: 0:09:34



time needed for map_ξ_GNC_multipole for auto_lensing [in s] = 574.61664 



In [17]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L2_withF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = true, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

auto_newton, L=2: 100%|█████████████████████████████████| Time: 0:00:13
auto_doppler, L=2: 100%|████████████████████████████████| Time: 0:00:13
auto_lensing, L=2:  41%|█████████████▎                  |  ETA: 0:05:41

LoadError: InterruptException: